In [71]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [72]:
df = pd.read_csv(r'C:\Users\yahoo net\Desktop\DataSets\netflix_data_1.txt', names = ['Cust_ID','Rating'],
                usecols = [0,1])

print(df.shape)

(24058263, 2)


In [73]:
print(df.head(10))

   Cust_ID  Rating
0       1:     NaN
1  1488844     3.0
2   822109     5.0
3   885013     4.0
4    30878     4.0
5   823519     3.0
6   893988     3.0
7   124105     4.0
8  1248029     3.0
9  1842128     4.0


In [74]:
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

In [75]:
start = 0
movie_id = 0
indeces = df_nan['index']
df['Movie_ID'] = np.zeros(len(df))
for i in indeces:
    df['Movie_ID'][start:i] = movie_id
    movie_id += 1
    start = i
df['Movie_ID'][df['Movie_ID'] == 0] = movie_id

df = df.dropna()

df['Cust_ID'] = df['Cust_ID'].astype(int)
df['Movie_ID'] = df['Movie_ID'].astype(int)

print(df.shape)

(24053764, 3)


In [76]:
print(df.iloc[::5000000, :])

          Cust_ID  Rating  Movie_ID
1         1488844     3.0         1
5000996    501954     2.0       996
10001962   404654     5.0      1962
15002876   886608     2.0      2876
20003825  1193835     2.0      3825


In [77]:
simple_math = ['count','mean']

df_movie_summary = df.groupby('Movie_ID')['Rating'].agg(simple_math)
print(df_movie_summary)

          count      mean
Movie_ID                 
1           547  3.749543
2           145  3.558621
3          2012  3.641153
4           142  2.739437
5          1140  3.919298
...         ...       ...
4495        614  3.478827
4496       9519  3.763000
4497        714  2.715686
4498        269  2.464684
4499        428  2.614486

[4499 rows x 2 columns]


In [78]:
minimum_movie_reviews = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < minimum_movie_reviews].index

print('Movie mininum times of reviews:', minimum_movie_reviews)

Movie mininum times of reviews: 1799.0


In [79]:
df_cust_summary = df.groupby('Cust_ID')['Rating'].agg(simple_math)
print(df_cust_summary)

         count      mean
Cust_ID                 
6          153  3.333333
7          195  4.066667
8           21  4.000000
10          49  3.346939
25           4  3.500000
...        ...       ...
2649404     12  4.083333
2649409     10  4.000000
2649421      3  5.000000
2649426     74  4.067568
2649429     62  4.161290

[470758 rows x 2 columns]


In [80]:
minimum_cust_reviews = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < minimum_cust_reviews].index

print(minimum_cust_reviews)

52.0


In [81]:
print('Old Shape:', df.shape)
df = df[~df['Movie_ID'].isin(drop_movie_list)]
df = df[~df['Cust_ID'].isin(drop_cust_list)]
print('Current Shape:', df.shape)
print('\n Data Ex:')
print(df.iloc[::5000000, :])

Old Shape: (24053764, 3)
Current Shape: (17337458, 3)

 Data Ex:
          Cust_ID  Rating  Movie_ID
696        712664     5.0         3
6932490   1299309     5.0      1384
13860273   400155     3.0      2660
20766530   466962     4.0      3923


In [94]:
titles = pd.read_csv(r'C:\Users\yahoo net\Desktop\DataSets\netflix_movie_titles.csv',
                     names = ['Movie_ID', 'Year', 'Movie_Name'])
titles = titles.dropna()
titles['Movie_ID'] = titles['Movie_ID'].astype(int)
print(titles.shape)

(17761, 3)


In [95]:
print(titles.head(10))

   Movie_ID    Year                    Movie_Name
0         1  2003.0               Dinosaur Planet
1         2  2004.0    Isle of Man TT 2004 Review
2         3  1997.0                     Character
3         4  1994.0  Paula Abdul's Get Up & Dance
4         5  2004.0      The Rise and Fall of ECW
5         6  1997.0                          Sick
6         7  1992.0                         8 Man
7         8  2004.0    What the #$*! Do We Know!?
8         9  1991.0      Class of Nuke 'Em High 2
9        10  2001.0                       Fighter


In [15]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

reader = Reader()

# get first 100000 rows to minimize time.
data = Dataset.load_from_df(df[['Cust_ID', 'Movie_ID', 'Rating']][0:100000], reader)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.9942647 , 1.00178707, 0.98933732, 1.00218281, 0.99935089]),
 'test_mae': array([0.80305265, 0.81097946, 0.79987314, 0.80990471, 0.79272972]),
 'fit_time': (4.920754432678223,
  4.311481714248657,
  4.342724561691284,
  4.436438083648682,
  4.467660665512085),
 'test_time': (0.1874556541442871,
  0.09372711181640625,
  0.18746042251586914,
  0.18746185302734375,
  0.10934925079345703)}

In [42]:
cust = df[(df['Cust_ID'] == 785314) & (df['Rating'] == 5)]
cust = cust.dropna()
cust['Movie_ID'] = cust['Movie_ID'].astype(int)
cust = cust.merge(titles, on = 'Movie_ID')

print(cust[['Cust_ID','Movie_ID','Movie_Name']])

cust = cust[['Movie_ID','Movie_Name']]
cust = cust[~cust['Movie_ID'].isin(drop_movie_list)]
temp = cust

    Cust_ID  Movie_ID                                         Movie_Name
0    785314        57                                        Richard III
1    785314       175                                     Reservoir Dogs
2    785314       311                                            Ed Wood
3    785314       329                                              Dogma
4    785314       331                                        Chasing Amy
5    785314       395                                      Captain Blood
6    785314       788                                             Clerks
7    785314       798                                               Jaws
8    785314       907                                    Animal Crackers
9    785314       985                                          The Mummy
10   785314      1552                                        Black Adder
11   785314      1905  Pirates of the Caribbean: The Curse of the Bla...
12   785314      2000                        Four W

In [17]:
data = Dataset.load_from_df(df[['Cust_ID', 'Movie_ID', 'Rating']], reader)

train_set = data.build_full_trainset()
svd.fit(train_set)

In [47]:
def Recommendation_Cust_ID(cust_id):
    global cust
    cust['Estimate_Score'] = cust['Movie_ID'].apply(lambda x: svd.predict(cust_id, x).est)
    cust = cust.drop(['Movie_ID'], axis = 1)

    cust = cust.sort_values('Estimate_Score', ascending = False)
    final = cust
    final = final.reset_index()
    cust = temp
    return final.head(10)

In [48]:
Recommendation_Cust_ID(785314)

,index,Movie_Name,Estimate_Score
0,43,Army of Darkness,4.929957
1,25,Evil Dead 2: Dead by Dawn,4.883672
2,10,Black Adder,4.845161
3,24,Monty Python: The Life of Python,4.814268
4,8,Animal Crackers,4.627970
5,34,Terminator 2: Extreme Edition: Bonus Material,4.530050
6,19,This Is Spinal Tap,4.477518
7,28,Spirited Away,4.448709
8,5,Captain Blood,4.443658
9,1,Reservoir Dogs,4.404526


In [49]:
Recommendation_Cust_ID(125489)

,index,Movie_Name,Estimate_Score
0,35,Finding Nemo (Widescreen),4.316222
1,18,Lord of the Rings: The Fellowship of the Ring,4.292432
2,23,The Silence of the Lambs,4.174110
3,32,The Sting,4.162566
4,40,The Sixth Sense,4.139118
5,28,Spirited Away,4.132371
6,11,Pirates of the Caribbean: The Curse of the Bla...,4.125807
7,1,Reservoir Dogs,4.117151
8,15,Singin' in the Rain,4.076795
9,44,To Have and Have Not,4.037314


In [50]:
Recommendation_Cust_ID(856974)

,index,Movie_Name,Estimate_Score
0,35,Finding Nemo (Widescreen),4.316222
1,18,Lord of the Rings: The Fellowship of the Ring,4.292432
2,23,The Silence of the Lambs,4.174110
3,32,The Sting,4.162566
4,40,The Sixth Sense,4.139118
5,28,Spirited Away,4.132371
6,11,Pirates of the Caribbean: The Curse of the Bla...,4.125807
7,1,Reservoir Dogs,4.117151
8,15,Singin' in the Rain,4.076795
9,44,To Have and Have Not,4.037314


In [108]:
df_p = pd.pivot_table(full_df, index = 'Cust_ID', columns = 'Movie_ID', values = 'Rating')


print(df_p.shape)

(143458, 1350)


In [109]:
def Recommendation_Movies(movie_name, min_count):
    index = int(titles.index[titles['Movie_Name'] == movie_name][0])
    target = df_p[index]
    similar_to_target = df_p.corrwith(target)
    corr_with_target = pd.DataFrame(similar_to_target, columns = ['Correlation'])
    corr_with_target.dropna(inplace = True)
    corr_with_target = corr_with_target.sort_values('Correlation', ascending = False)
#     corr_with_target.index = corr_with_target.index.map(int)
    corr_with_target = corr_with_target.join(titles).join(df_movie_summary)[['Correlation', 'Movie_Name', 'count']]
    print(corr_with_target[corr_with_target['count'] > min_count][:10])

In [110]:
Recommendation_Movies("Without a Trace: Season 1", 0)

          Correlation                          Movie_Name  count
Movie_ID                                                        
3926         1.000000           Without a Trace: Season 1  13359
3775         0.408262         In the Mirror of Maya Deren   1987
4092         0.396701  Daydream Obsession 2: Infidelities   2226
341          0.385631                                Noir   1862
297          0.384641      Crunch: Pick Your Spot Pilates   1956
4173         0.377569               Don't Drink the Water   1984
1089         0.370469                                 Eel   2083
1455         0.370263                  Nina Takes a Lover   3859
488          0.368137                  Return to Paradise   3247
2988         0.353877        The Thing From Another World   3234


In [112]:
Recommendation_Movies("Return to Paradise", 0)

          Correlation                                         Movie_Name  \
Movie_ID                                                                   
488          1.000000                                 Return to Paradise   
4238         0.783164                                          Maitresse   
1766         0.764925                      The Gold Rush: Bonus Material   
1607         0.741386  Silverlake Life: The View From Here: 10th Anni...   
3927         0.738947                                 Nip/Tuck: Season 2   
3593         0.734872                            The Old Man and the Sea   
2405         0.716314                         A Touch of Frost: Season 2   
2019         0.715837                                   The Winds of War   
774          0.706759                Man Ray: Prophet of the Avant Garde   
180          0.702771                                      The Last Shot   

          count  
Movie_ID         
488        3247  
4238       1883  
1766       2327

In [114]:
Recommendation_Movies("Richard III", 0)

          Correlation                        Movie_Name  count
Movie_ID                                                      
56           1.000000                       Richard III   1833
575          0.853486                 Popular: Season 2   2122
2568         0.778990                Washington Heights   1812
371          0.770824                             Jeans   2004
477          0.769802           The Beverly Hillbillies   2241
1179         0.768520                  A Beautiful Mind   1913
104          0.768228                          Obsessed   3500
579          0.753169         THX 1138: Special Edition   2047
4383         0.737642                  Dawn of the Dead   4051
3229         0.733387  The Best of the Red Skelton Show   1938
